In [ ]:
import cv2
import csv
import os
import numpy as np

def get_pixel_length(image):
    """
    Permite al usuario seleccionar dos puntos en la imagen para calcular la longitud en píxeles entre ellos.
    """
    points = []
    image_copy = image.copy()

    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            cv2.circle(image_copy, (x, y), 3, (0, 255, 0), -1)
            cv2.imshow('image', image_copy)

    cv2.imshow('image', image_copy)
    cv2.setMouseCallback('image', mouse_callback)

    while len(points) < 2:
        cv2.waitKey(1)

    cv2.destroyAllWindows()

    if len(points) == 2:
        return np.sqrt((points[1][0] - points[0][0])**2 + (points[1][1] - points[0][1])**2)
    else:
        return None

def calculate_mm_per_pixel(pixel_length, scale_mm):
    """
    Calcula los milímetros por píxel dados una longitud en píxeles y la longitud real en milímetros.
    """
    return pixel_length / scale_mm

def save_to_csv(data, csv_file):
    """
    Guarda los datos en un archivo CSV.
    """
    file_exists = os.path.exists(csv_file)
    with open(csv_file, 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)
    print(f"Datos guardados en {csv_file}")

def process_images_in_directory(directory_path, scale_mm, csv_file='escala.csv'):
    """
    Procesa todas las imágenes en un directorio para calcular y guardar la escala.
    """
    for image_name in os.listdir(directory_path):
        image_path = os.path.join(directory_path, image_name)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            print(f"Procesando imagen: {image_name}")
            image = cv2.imread(image_path)
            if image is None:
                print(f"No se pudo cargar la imagen {image_name}.")
                continue

            print("Por favor, haga clic en el inicio y el final de la línea de escala de 3mm.")
            pixel_length = get_pixel_length(image)

            if pixel_length is None:
                print(f"No se pudo calcular la longitud en píxeles para {image_name}.")
                continue

            print(f"La línea de escala mide {pixel_length:.2f} píxeles.")
            mm_per_pixel = calculate_mm_per_pixel(pixel_length, scale_mm)
            print(f"Cada píxel representa {mm_per_pixel:.4f} mm.")

            data = {
                'imagen': image_name,
                'pixeles': pixel_length,
                'mm_por_pixel': mm_per_pixel
            }
            save_to_csv(data, csv_file)

# Configuración inicial
directory_path = '../imagenes/seleccionadas/medio'
scale_mm = 3  # Longitud de la escala en milímetros
csv_file = 'escala.csv'

# Procesar todas las imágenes en el directorio
process_images_in_directory(directory_path, scale_mm, csv_file)
